In [80]:
import mne
import numpy as np
import matplotlib as mp
import pandas as pd
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import read_layout
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
from mne.time_frequency import tfr_multitaper
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.viz.utils import center_cmap
from mne.io import concatenate_raws

import os
import glob

In [81]:
frame=pd.DataFrame()
dataarray=np.zeros([10,14,15000])# i changed this value from 15000
df=pd.DataFrame()
basepath='S05/day2'
iter=0
for trail in os.listdir(os.path.join(basepath)):
    for file in  os.listdir(os.path.join(basepath,trail)):
        if(trail[:-2]=='neurofeedback' and (int(trail[-2:])<=5 or int(trail[-2:])>=11)):
            df=pd.read_csv(os.path.join(basepath,trail,file),sep='\t',header=None)
            df=df.T
            print(basepath,trail,file)
            print(df.shape)
            data=df.values;
            dataarray[iter,:,:]=data[:,0:15000] #  i changed this value 15000
            iter+=1
dataarray.shape

S05/day2 neurofeedback01 anodal_S05_day2_nf01_EEGraw_and_condition.txt
(14, 18000)
S05/day2 neurofeedback02 anodal_S05_day2_nf02_EEGraw_and_condition.txt
(14, 19024)
S05/day2 neurofeedback03 anodal_S05_day2_nf03_EEGraw_and_condition.txt
(14, 19520)
S05/day2 neurofeedback04 anodal_S05_day2_nf04_EEGraw_and_condition.txt
(14, 19696)
S05/day2 neurofeedback05 anodal_S05_day2_nf05_EEGraw_and_condition.txt
(14, 17280)
S05/day2 neurofeedback11 anodal_S05_day2_nf11_EEGraw_and_condition.txt
(14, 18272)
S05/day2 neurofeedback12 anodal_S05_day2_nf12_EEGraw_and_condition.txt
(14, 19744)
S05/day2 neurofeedback13 anodal_S05_day2_nf13_EEGraw_and_condition.txt
(14, 19344)
S05/day2 neurofeedback14 anodal_S05_day2_nf14_EEGraw_and_condition.txt
(14, 19680)
S05/day2 neurofeedback15 anodal_S05_day2_nf15_EEGraw_and_condition.txt
(14, 18880)


(10, 14, 15000)

In [3]:
# data=np.load('./anodal/S03day1.npy')
# cols=['Fcz', 'Fc2', 'Fc4', 'Fc6', 'Cz', 'C2', 'C4', 'C6', 'Cpz', 'Cp2', 'Cp4', 'Cp6','pz','stim']
# #df=pd.DataFrame(data=data.T,,columns=cols)
# print(data.shape)
# datastack=np.reshape(data,(10*14,18000))
# #data=np.delete(data,[7],0)
# #df=pd.DataFrame(data=datastack)
#df.head(14)

In [82]:
%matplotlib
dataarray[:,:13,:]=dataarray[:,:13,:]/1000000
data=dataarray
[ntrail,nchann,nsamp]=data.shape


sampling_freq = 128  # in Hertz
ch_names =['F4', 'F8', 'C4', 'T4', 'Cz', 'C2', 'T6', 'C6', 'CPz', 'CP2', 'CP4', 'CP6','A1','STI 001']
#ch_names = ['FCz', 'FC2', 'FC4', 'FC6', 'Cz', 'C2', 'C4', 'C6', 'CPz', 'CP2', 'CP4', 'CP6','A1','STI 001']
ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg','eeg','stim']

info = mne.create_info(ch_names=ch_names, sfreq=sampling_freq, ch_types=ch_types)

raw= mne.io.RawArray(data[6,:,:], info)#**change this value one by one, Save Figure 3 ONLY for Pre MI 0 to 4, for Post MI,5 t0 9
raw.filter(l_freq=3, h_freq=40) #filter the fraw data from 8 to 22 hertz
raw.set_montage('standard_1020')
raw.plot();
#raw = concatenate_raws([mne.io.RawArray(data[f,:,:], info) for f in range(0,ntrail)])
#    raw=mne.io.concatenate_raws([raw, mne.io.RawArray(data[1,:,:], info)])

# ica = mne.preprocessing.ICA(n_components=13, random_state=97, max_iter=800)
# ica.fit(raw)
# ica.plot_sources(raw, show_scrollbars=True);
# ica.plot_components(title='compnents',cmap='jet')

# ica.plot_properties(raw, picks=[0,1,2,3,4,5,6,7,8, 9,10,11])

# print(raw.info)
# #raw.filter(l_freq=12, h_freq=22)
# # raw.set
                             
events = mne.find_events(raw, 'STI 001')
print(events)


Using matplotlib backend: Qt5Agg
Creating RawArray with float64 data, n_channels=14, n_times=15000
    Range : 0 ... 14999 =      0.000 ...   117.180 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 3.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 213 samples (1.664 sec)

23 events found
Event IDs: [1 2 3]
[[  256     0     1]
 [  384     1     2]
 [ 1392     2     3]
 [ 2288     0     1]
 [ 2416     1     2]
 [ 2848     2     3]
 [ 3744     0     1]
 [ 3872     1     2]
 [ 4880     2     3]
 [ 5776     0     1]
 [ 5904     1     2]
 [ 6912  

In [83]:

raw.info['bads'] += [ 'A1']
picks = mne.pick_types(raw.info, eeg=True, stim=False,exclude='bads')
picks = mne.pick_channels(raw.info["ch_names"], ['F4', 'F8','C4', 'T4',])
tmin, tmax = -3, 8  # define epochs around events (in s)
event_ids = {'ready': 1, 'MI': 2, 'postrest': 3}  # map event IDs to tasks
reject= dict(  eeg=150e-6  )     # 150 µV
epochs = mne.Epochs(raw, events, event_ids, tmin - 0.5, tmax + 0.5,
                    picks=picks,baseline=None, preload=True)


# ica = mne.preprocessing.ICA(n_components=12, random_state=97, max_iter=800)
# ica.fit(epochs)
# ica.plot_sources(epochs, show_scrollbars=True);
# ica.plot_components(title='compnents',cmap='jet')



freqs = np.arange(8, 24, 1)  # frequencies from 2-35Hz
n_cycles = freqs  # use constant t/f resolution
vmin, vmax = -1, 1.5  # set min and max ERDS values in plot
baseline = [-3, 0]  # baseline interval (in s)
cmap = center_cmap(plt.cm.RdBu, vmin, vmax)  # zero maps to white
kwargs = dict(n_permutations=100, step_down_p=0.05, seed=1,
              buffer_size=None, out_type='mask')  # for cluster test

# Run TF decomposition overall epochs
tfr = tfr_multitaper(epochs, freqs=freqs, n_cycles=n_cycles,
                     use_fft=True, return_itc=False, average=False,
                     decim=2)
tfr.crop(tmin, tmax)
tfr.apply_baseline(baseline, mode="percent")
for event in event_ids:
    # select desired epochs for visualization
    tfr_ev = tfr[event]
    fig, axes = plt.subplots(1,5,figsize=(12, 4), gridspec_kw={"width_ratios": [10, 10, 10,10 ,1]})
      
    for ch, ax in enumerate(axes[:-1]):  # for each channel
        # positive clusters
        _, c1, p1, _ = pcluster_test(tfr_ev.data[:, ch, ...], tail=1, **kwargs)
        # negative clusters
        _, c2, p2, _ = pcluster_test(tfr_ev.data[:, ch, ...], tail=-1,
                                     **kwargs)

        # note that we keep clusters with p <= 0.05 from the combined clusters
        # of two independent tests; in this example, we do not correct for
        # these two comparisons
        c = np.stack(c1 + c2, axis=2)  # combined clusters
        p = np.concatenate((p1, p2))  # combined p-values
        mask = c[..., p <= 0.05].any(axis=-1)

        # plot TFR (ERDS map with masking)
        tfr_ev.average().plot([ch], vmin=vmin, vmax=vmax, cmap=(cmap, False),
                              axes=ax, colorbar=False, show=False, mask=mask,
                              mask_style="mask")

        ax.set_title(epochs.ch_names[ch], fontsize=10)
        ax.axvline(0, linewidth=1, color="black", linestyle=":")  # event
        if ch != 0:
            ax.set_ylabel("")
            ax.set_yticklabels("")
    fig.colorbar(axes[0].images[-1], cax=axes[-1])
    fig.suptitle("ERDS ({})".format(event))
    fig.show()

Not setting metadata
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Loading data for 23 events and 1537 original time points ...
3 bad epochs dropped
Not setting metadata
Applying baseline correction (mode: percent)
Using a threshold of 1.943180
stat_fun(H1): min=-74.476079 max=7.283782
Running initial clustering
Found 10 clusters
Permuting 99 times...



Computing cluster p-values
Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
Done.
Using a threshold of -1.943180
stat_fun(H1): min=-74.476079 max=7.283782
Running initial clustering
Found 45 clusters
Permuting 99 times...



Computing cluster p-values
Step-down-in-jumps iteration #1 found 1 cluster to exclude from subsequent iterations
Permuting 99 times...



Computing cluster p-values
Step-down-in-jumps iteration #2 found 0 additional clusters to exclude from subsequent iterations
Done.
No baseline correction applied
Using a threshold of 1.943180
stat_fun(H1): min=-74.227490 max=5.134566
Running initial clustering
Found 13 clusters
Permuting 99 times...



Computing cluster p-values
Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
Done.
Using a threshold of -1.943180
stat_fun(H1): min=-74.227490 max=5.134566
Running initial clustering
Found 44 clusters
Permuting 99 times...


...


Computing cluster p-values
Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
Done.
No baseline correction applied
Using a threshold of 1.943180
stat_fun(H1): min=-5.532341 max=6.416746
Running initial clustering
Found 63 clusters
Permuting 99 times...



Computing cluster p-values
Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
Done.
Using a threshold of -1.943180
stat_fun(H1): min=-5.532341 max=6.416746
Running initial clustering
Found 20 clusters
Permuting 99 times...



Computing cluster p-values
Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
Done.
No baseline correction applied
Using a threshold of 1.943180
stat_fun(H1): min=-5.544061 max=4.843052
Running initial clustering
Found 76 clusters
Permuting 99 times...



Computing cluster p-values
Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
Done.
Using a threshold of -1.943180
stat_fun(H1): min=-5.544061 max=4.843052
Running initial clustering
Found 21 clusters
Permuting 99 times...



Computing cluster p-values
Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
Done.
No baseline correction applied
